In [1]:
import pandas as pd
import datetime
import os

In [2]:
df_sii = pd.read_excel('facturas electronicas 06.09.2022.xlsx')
df_exentas = pd.read_excel('facturas exentas 06.09.2022.xlsx')
df_presupuesto = pd.read_excel('presupuesto 06.09.2022.xlsx')
df_sci = pd.read_excel('sci 06.09.2022.xlsx')
df_acepta = pd.read_excel('acepta_2022.xlsx')

df_presupuesto.rename(columns = {'Rut': 'RUT Emisor', 'Folio': 'Folio_abreviado', 'NºDoc.': 'Folio'}, inplace = True)
df_sci.rename(columns = {'Rut Proveedor': 'RUT Emisor', 'Numero Documento': 'Folio'}, inplace = True)
df_acepta.rename(columns = {'emisor': 'RUT Emisor', 'folio': 'Folio'}, inplace = True)

In [3]:
todas_las_df = {'SII': df_sii, 'EXENTAS': df_exentas, 'PRESUPUESTO': df_presupuesto, 'SCI': df_sci, 'ACEPTA': df_acepta}
for nombre_tabla, df in todas_las_df.items():
    df.columns = df.columns + f' {nombre_tabla}'
    df['llave_id'] = df[f'RUT Emisor {nombre_tabla}'].astype(str) + df[f'Folio {nombre_tabla}'].astype(str)
    df.set_index('llave_id', drop = True, inplace = True)

In [9]:
df_sii_exentas = pd.merge(df_sii, df_exentas, how = 'left', left_index = True, right_index = True)
df_sii_exentas_presupuesto = pd.merge(df_sii_exentas, df_presupuesto, how = 'left', left_index = True, right_index = True)
df_sii_exentas_presupuesto_sci = pd.merge(df_sii_exentas_presupuesto, df_sci, how = 'left', left_index = True, right_index = True)
df_sii_exentas_presupuesto_sci_acepta = pd.merge(df_sii_exentas_presupuesto_sci, df_acepta, how = 'left', left_index = True, right_index = True)

df_sii_exentas_presupuesto_sci_acepta = df_sii_exentas_presupuesto_sci_acepta[~df_sii_exentas_presupuesto_sci_acepta.index.duplicated(keep = 'first')] 

df_sii_exentas_presupuesto_sci_acepta['tiempo_diferencia'] = pd.to_datetime('today') - df_sii_exentas_presupuesto_sci_acepta['Fecha Docto. SII']
df_sii_exentas_presupuesto_sci_acepta['esta_al_dia'] = df_sii_exentas_presupuesto_sci_acepta['tiempo_diferencia'] <= datetime.timedelta(8)

columnas_a_ocupar = ['Nro. SII', 'RUT Emisor SII', 'razon_social_emisor ACEPTA', 'folio_oc ACEPTA', 'Folio SII', 'Fecha Docto. SII', 'Monto Neto SII', 
                     'Monto Exento SII', 'Monto IVA SII', 'Monto Total SII', 'Fecha Recep. SII', 'Ubic. PRESUPUESTO',
                     'tarea_actual ACEPTA', 'estado_devengo ACEPTA', 'fecha_ingreso_rc ACEPTA', 'estado_nar ACEPTA', 'Motivo SCI', 'tiempo_diferencia', 'esta_al_dia', 'Evento Receptor SII']

df_columnas_utiles = df_sii_exentas_presupuesto_sci_acepta[columnas_a_ocupar]


# df_sii_exentas_presupuesto_sci_acepta.to_excel('MATRIZ_FINAL_COMPLETA.xlsx')
fecha_actual = str(pd.to_datetime('today')).split(' ')[0]
nombre_archivo = f'PLANILLA DE CONTROL AL {fecha_actual}.xlsx'

if nombre_archivo in os.listdir():
    with pd.ExcelWriter(nombre_archivo, engine = 'openpyxl', mode = 'a', if_sheet_exists = 'overlay') as writer:
        df_columnas_utiles.to_excel(writer)
else:
    df_columnas_utiles['Observaciones'] = None
    with pd.ExcelWriter(nombre_archivo, engine = 'openpyxl', mode = 'w') as writer:
        df_columnas_utiles.to_excel(writer)

C:\Users\gmolina\AppData\Local\Temp\ipykernel_16696\1105263362.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_columnas_utiles['Observaciones'] = None
